# 自然语言处理第二次作业

王逸群 19307110397

## 0. 准备工作

### 0.1 数据导入

数据集来源于Wikipedia，存储于`data.json`，包含10个类别10000个样本。

每个样本都是一个字典，包含三个键：`title`、`label`和`text`。

In [1]:
import json

In [2]:
dat = []
with open("data.json", "r") as file:
    for line in file:
        dat.append(json.loads(line))

### 0.2 数据预处理

在`dat`中增加两个键，存储样本中的句子和单词。

对每句话，保留英语单词和数字，去除标点符号和特殊字符。

其中，英语单词全都小写。

In [3]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

In [4]:
for sample in dat:
    sample['sent'] = [[word.lower() for word in word_tokenize(sent) if word.isdigit() or word.isalpha()] 
                      for sent in sent_tokenize(sample['text'])]
    sample['word'] = [word for sent in sample['sent'] for word in sent]

另外，生成句子列表和词汇集合：

In [ ]:
sents = [sent for sample in dat for sent in sample['sent'] if sent != []]
words = set([word for sent in sents for word in sent])

## 1. 词汇编码

以数据集作为训练集，使用`gensim`工具，生成SGNS词汇编码模型。

其中，超参数使用默认参数：

编码维度：`vector_size = 100`；

窗口大小：`window = 5`；

负采样参数：`negative = 5`；

初始学习率：`alpha = 0.025`：

In [8]:
from gensim.models import Word2Vec

In [ ]:
word2vec = Word2Vec(sents, min_count = 1, epochs = 1)

为了方便后续使用，保存模型：

In [ ]:
word2vec.save('word2vec.model')

后续使用以下代码载入模型：

In [16]:
word2vec = Word2Vec.load('word2vec.model')

## 2. 词汇相似度

随机生成100、1000、10000对词汇，对每个词汇集合，以余弦相似度作为度量，打印5对最相似和5对最不相似的词汇。

In [25]:
import random
import numpy as np
from numpy import linalg as LA

In [68]:
random.seed(0)

In [69]:
for num in (100, 1000, 10000):
    word_similar = []
    word_unsimilar = []
    dist_similar = []
    dist_unsimilar = []
    
    for _ in range(num):
        word_sample = random.sample(words, 2)
        word_vector = [word2vec.wv[word_sample[0]], word2vec.wv[word_sample[1]]]
        dist = np.dot(word_vector[0], word_vector[1]) / (LA.norm(word_vector[0]) * LA.norm(word_vector[1]))
            
        if len(dist_similar) < 5:
            word_similar.append([word_sample[0], word_sample[1]])
            dist_similar.append(dist)
        elif dist > min(dist_similar):
            index = dist_similar.index(min(dist_similar))
            word_similar[index] = [word_sample[0], word_sample[1]]
            dist_similar[index] = dist
            
        if len(dist_unsimilar) < 5:
            word_unsimilar.append([word_sample[0], word_sample[1]])
            dist_unsimilar.append(dist)
        elif dist < max(dist_unsimilar):
            index = dist_unsimilar.index(max(dist_unsimilar))
            word_unsimilar[index] = [word_sample[0], word_sample[1]]
            dist_unsimilar[index] = dist
    
    print('Within', num, 'word pairs, the most similar and unsimilar words:')
    print('words\t\t\t\t\tdistance')
    for index in range(5):
        print('%-20s%-20s%8.5f' % (word_similar[index][0], word_similar[index][1], dist_similar[index]))
    for index in range(5):
        print('%-20s%-20s%8.5f' % (word_unsimilar[index][0], word_unsimilar[index][1], dist_unsimilar[index]))
    print()

Within 100 word pairs, the most similar and unsimilar words:
words					distance
wistrom             bly                  0.67645
darwaza             lombardi             0.64596
athmanathan         kindliness           0.52732
kinase              vada                 0.73586
gori                zelimkhan            0.61509
kaiciid             trickiest           -0.15902
kamaras             kallikak            -0.13632
ackaratorn          vidhamba            -0.19035
lanús               garlicka            -0.16384
leachable           jacovleff           -0.13062

Within 1000 word pairs, the most similar and unsimilar words:
words					distance
pura                lunaspis             0.75509
kochuri             htc                  0.72150
streamlining        kandinsky            0.69936
venus               sy                   0.74617
hezhong             gadot                0.69031
approvingalthough   overuse             -0.28227
kuosheng            cocea               -0.34014
hasim

可以看到，

## 3. 文档编码

分别使用词汇编码均值、第一段词汇编码均值、文档编码算法三种方法进行文档编码，

接着使用k均值聚类算法、计算微观和宏观F1值评价各方法，并使用t-SNE进行可视化。

### 3.1 使用词汇编码均值

### 3.2 使用第一段词汇编码均值

### 3.3 使用文档编码算法

In [ ]:
from gensim.models.doc2vec import TaggedDocument, Doc2Vec